In [33]:
import pandas as pd
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import torch.nn.functional as F
from torch.optim import Adam
from sklearn.model_selection import train_test_split
from data import load_qm9, split
from models import LinReg, E3EGNN, MLP
from train_eval import train_model, evaluate_model
import numpy as np
from tqdm import tqdm
from torch_geometric.datasets import QM9

ImportError: cannot import name 'LinReg2' from 'models' (/Users/samsongourevitch/Documents/EGGN_Classifier/src/models.py)

In [30]:
import sys
sys.path.append('/Users/samsongourevitch/Documents/EGGN_Classifier/src/')


In [32]:
import sys
print(sys.path)

['/Users/samsongourevitch/.pyenv/versions/3.8.10/lib/python38.zip', '/Users/samsongourevitch/.pyenv/versions/3.8.10/lib/python3.8', '/Users/samsongourevitch/.pyenv/versions/3.8.10/lib/python3.8/lib-dynload', '', '/Users/samsongourevitch/.pyenv/versions/3.8.10/lib/python3.8/site-packages', '/Users/samsongourevitch/Documents/EGGN_Classifier/src/']


In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
qm9_dataset = load_qm9()

Processing...
100%|██████████| 133885/133885 [00:53<00:00, 2512.45it/s]
Done!
/Users/samsongourevitch/.pyenv/versions/3.8.10/lib/python3.8/site-packages/torch_geometric/data/in_memory_dataset.py:300: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


In [15]:
# Split dataset into train val and test sets
train_dataset, val_dataset, test_dataset = split(qm9_dataset)

In [ ]:
# Define the models
linreg = LinReg().to(device)
mlp = MLP().to(device)
e3_model = E3EGNN().to(device)
# se3_model = SE3EGNN().to(device)

# Define optimizers and loss function
optimizer_e3 = Adam(e3_model.parameters(), lr=0.001)
# optimizer_se3 = Adam(se3_model.parameters(), lr=0.001)

In [ ]:
# Train and evaluate
epochs = 10
for epoch in range(epochs):
    train_model(e3_model, train_loader, optimizer_e3)
    # train_model(se3_model, train_loader, optimizer_se3)
    train_loss_linreg, train_acc_linreg = evaluate_model(linreg, train_dataset)
    train_loss_mlp, train_acc_mlp = evaluate_model(mlp, train_loader)
    train_loss_e3, train_acc_e3 = evaluate_model(e3_model, train_dataset)
    # train_loss_se3, train_acc_se3 = evaluate_model(se3_model, train_loader)
    test_loss_e3, test_acc_e3 = evaluate_model(e3_model, test_dataset)
    # test_loss_se3, test_acc_se3 = evaluate_model(se3_model, test_loader)
    print(f"[Epoch {epoch+1}] Train Acc (E3): {train_acc_e3:.4f}, Test Acc (E3): {test_acc_e3:.4f}")
